In [129]:
import gensim
import os
import json
import tqdm
import time
import numpy as np
import random
import nltk
import math



from collections import Counter
from gensim.models.word2vec import Word2Vec
from sys import getsizeof

In [150]:
ts = time.strftime('%Y-%b-%d-%H-%M-%S', time.gmtime())

save_model_path = os.path.join('won_seq2seq', ts)
os.makedirs('.\\'+save_model_path)

In [3]:
with open('dict_targets_Reviews.json', 'r', encoding='UTF-8') as f:
    target_dict=json.load(f)

In [4]:
with open('dict_inputs_Reviews.json', 'r', encoding='UTF-8') as f:
    inputs_dict=json.load(f)

In [5]:
max_length = 100

In [6]:
with open('word_unk_dic.json', 'r', encoding='UTF-8') as f:
    word_unk_dic=json.load(f)

word_list = word_unk_dic['WORD']

unk_list = word_unk_dic['UNK']

In [7]:
_PAD_ = "_PAD_"  # 빈칸 채우는 심볼
_STA_ = "_STA_"  # 디코드 입력 시퀀스의 시작 심볼
_EOS_ = "_EOS_"  # 디코드 입출력 시퀀스의 종료 심볼
_UNK_ = "_UNK_"  # 사전에 없는 단어를 나타내는 심볼

_PAD_ID_ = 0
_STA_ID_ = 1
_EOS_ID_ = 2
_UNK_ID_ = 3
_PRE_DEFINED_ = [_PAD_ID_, _STA_ID_, _EOS_ID_, _UNK_ID_]

_PRE_DEFINED_dict = {_PAD_:_PAD_ID_, _STA_:_STA_ID_, _EOS_:_EOS_ID_, _UNK_:_UNK_ID_}

In [8]:
def padding(seq, max_len, start=None, eos=None):
    if start:
        padded_seq = [_STA_] + seq
    elif eos:
        padded_seq = seq + [_EOS_]
    else:
        padded_seq = seq

    if len(padded_seq) < max_len:
        #print(len(padded_seq))
        padded_seq = padded_seq + ([_PAD_] * (max_len - len(padded_seq)))
        if len(padded_seq) < max_len:
            return padded_seq[:max_len - len(padded_seq)]
        return padded_seq
    else:
        if len(padded_seq) > max_len:
            return padded_seq[:-1]
        else : 
            return padded_seq

In [9]:
def unk_make(data):
    
    for index, word in enumerate(data):

        if word in word_list:
            pass
        else:
            data[index] = _UNK_
            pass
            
    return data

In [10]:
def data_padding(data,max_len = 0, start=None, eos=None):
    pos_padding_dict = {}
    for title in tqdm.tqdm(list(data.keys())):
        if (len(data[title])>max_len):
            data[title] = data[title][:max_len]
        sentence = unk_make(data[title])
        pos_padding_dict[title] = padding(sentence,max_len,start,eos)
        
    return pos_padding_dict

In [11]:
def dic_lists(data):
    dic_list = []
    for key in data.keys():
        dic_list.append(data[key])
    return dic_list

In [12]:
targets_pad = data_padding(target_dict,max_length,None,True)
inputs_pad = data_padding(inputs_dict,max_length,True,None)

100%|█████████████████████████████████████████████████████████████████████████| 568427/568427 [10:42<00:00, 884.17it/s]


In [13]:
inputs = dic_lists(inputs_pad)

In [14]:
targets = dic_lists(targets_pad)

In [15]:
for i in inputs:
    if len(i) != 100:
        print(len(i))

In [16]:
model = gensim.models.Word2Vec.load("./Word2vec_data.model")

In [170]:
import torch
from torch.autograd import Variable
import torch.nn.utils.rnn as rnn_utils
import torch.nn as nn
import torch.nn.functional as F

In [18]:
word2index=model.wv.index2word

word2index = {}
for i,j in enumerate(model.wv.index2word):
    word2index[j]=i
index2word=model.wv.index2word

model_embedding = []
for index in model.wv.index2word:
    model_embedding.append(model.wv[index])
model_embedding = np.array(model_embedding)

In [19]:
def word_2_index(datas):
    data_index = []
    for data in tqdm.tqdm(datas):
        temp = []
        for word in data:
            temp.append(word2index[word])
        data_index.append(temp)
    return data_index

In [20]:
target_index = word_2_index(targets)
input_index = word_2_index(inputs)

100%|███████████████████████████████████████████████████████████████████████| 568427/568427 [00:09<00:00, 57269.20it/s]


In [21]:
max_sequence_length = max_length
embedding_size = 300
hidden_size = 256
word_dropout = 0.5
num_layers = 1
bidirectional = True
batch_size_fit = 64
rnn_type = 'gru'
learning_rate = 0.0001
k = 0.002
x0 = 8000
vocab_size = len(index2word)
sos_idx = word2index['_STA_']
eos_idx = word2index['_EOS_']
pad_idx = word2index['_PAD_']

In [133]:
class Seq2Seq(nn.Module):
    def __init__(self,vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout,
                sos_idx, eos_idx, pad_idx, numpy_embedding,rnn_type='rnn' , num_layers=1, bidirectional=True):
        
        super().__init__()
        self.model_embedding = torch.from_numpy(numpy_embedding)
        self.max_sequence_length = max_sequence_length
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.embedding.weight = nn.Parameter(self.model_embedding)
        
        self.encoder = Encoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,rnn_type = rnn_type).cuda()
        self.decoder = Decoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,rnn_type = rnn_type, word_dropout=word_dropout).cuda()
    
    def forward(self,x,length): 
        #print(x.size())
        batch_size = x.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = x[sorted_idx.cuda()]
        input_embedding = self.embedding(input_sequence)
        
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.tolist(), batch_first=True)

        self.encoder_outputs, self.encoder_hidden_state = self.encoder(packed_input,batch_size)

        decoder_outputs  = self.decoder(input_embedding, self.encoder_outputs, self.encoder_hidden_state, sorted_lengths, sorted_idx,batch_size)
        
        
        return decoder_outputs

In [134]:
class Encoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, bidirectional=True, num_layers = 1,rnn_type='rnn'):
        super(Encoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        
        self.encoder = rnn(self.embedding_size, self.hidden_size, num_layers = self.num_layers, bidirectional = self.bidirectional, batch_first = True)
        
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers

    
    def forward(self,x,batch_size):
        
        outputs, hidden = self.encoder(x)
        
        if self.bidirectional or self.num_layers > 1:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size*self.hidden_factor)
        else:
            hidden = hidden.squeeze()

        
        return outputs, hidden

In [205]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.normal_(mean=0, std=stdv)

    def forward(self, hidden, encoder_outputs):
        '''
        :param hidden: 
            previous hidden state of the decoder, in shape (layers*directions,B,H)
        :param encoder_outputs:
            encoder outputs from Encoder, in shape (T,B,H)
        :return
            attention energies in shape (B,T)
        '''
        padded_outputs = rnn_utils.pad_packed_sequence(encoder_outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _,reversed_idx = torch.sort(sorted_idx.cuda())
        self.encoder_outputs = padded_outputs[reversed_idx]
        this_batch_size,max_len,_ = self.encoder_outputs.size()
 
        self.H = hidden.repeat(max_len,1,1).transpose(0,1)
        attn_energies = self.score(self.H,self.encoder_outputs) # compute attention score
        return F.softmax(attn_energies).unsqueeze(1) # normalize with softmax

    def score(self, hidden, encoder_outputs):
        energy = F.tanh(self.attn(torch.cat([hidden, encoder_outputs], 2))) # [B*T*2H]->[B*T*H]
        energy = energy.transpose(2,1) # [B*H*T]
        v = self.v.repeat(encoder_outputs.data.shape[0],1).unsqueeze(1) #[B*1*H]
        energy = torch.bmm(v,energy) # [B*1*T]
        return energy.squeeze(1) #[B*T]

In [202]:
attn=Attn('concat', hidden_size).cuda()

In [217]:
attn_weight.bmm(attn.encoder_outputs)

Variable containing:
( 0 ,.,.) = 
 -7.5440e-03  3.8769e-02  2.7678e-02  ...  -1.9421e-02 -9.5482e-02  1.0714e-01

( 1 ,.,.) = 
 -4.0386e-02  4.4279e-02  1.0872e-01  ...  -3.8204e-02 -5.7070e-02  1.4929e-01

( 2 ,.,.) = 
 -1.4308e-02  2.1524e-02  2.5943e-02  ...   6.0177e-03 -5.0823e-02  4.7969e-02
... 

(61 ,.,.) = 
 -6.9729e-04  2.7002e-02  4.8029e-02  ...  -2.6745e-02 -1.9201e-02  6.1623e-02

(62 ,.,.) = 
 -1.7530e-02  1.9133e-02  3.6887e-02  ...  -7.6165e-03 -2.1013e-02  3.3945e-02

(63 ,.,.) = 
 -1.7988e-02  2.0094e-03  2.7185e-02  ...  -7.7520e-03 -1.8387e-02  2.1068e-02
[torch.cuda.FloatTensor of size 64x1x256 (GPU 0)]

In [207]:
attn_weight=attn(S2S.encoder_hidden_state.unsqueeze(0), S2S.encoder_outputs)

torch.Size([64, 100, 256])
torch.Size([64, 100, 256])


C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
class Decoder_attn(nn.Module):
    def __init__(self,vocab_size,embedding_size , hidden_size, bidirectional=True, num_layers = 1,rnn_type='rnn',word_dropout = 0.5):
        super(Decoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers            
        self.attn=Attn('concat', hidden_size)
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.word_dropout = nn.Dropout(p=word_dropout)
        
        self.decoder = rnn(embedding_size, hidden_size, num_layers=num_layers, bidirectional=self.bidirectional, batch_first=True)
        self.hidden2outputs = nn.Linear(hidden_size * (2 if bidirectional else 1), hidden_size)
        self.outputs2vocab = nn.Linear(hidden_size, vocab_size)
        

    def forward(self,x,encoder_outputs,hidden,sorted_lengths,sorted_idx,batch_size):

        
        if self.bidirectional or self.num_layers > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.hidden_size)
        else:
            hidden = hidden.unsqueeze(0)

        input_embedding = self.word_dropout(x)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.tolist(), batch_first=True)
        outputs,_ = self.decoder(packed_input, hidden)
        
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _,reversed_idx = torch.sort(sorted_idx.cuda())
        padded_outputs = padded_outputs[reversed_idx]
        b,s,_ = padded_outputs.size()
        
        #print(padded_outputs.view(-1, padded_outputs.size(2)).size())
        outputs = nn.functional.log_softmax(self.outputs2vocab(self.hidden2outputs(padded_outputs.view(-1, padded_outputs.size(2)))), dim=-1)
        outputs = outputs.view(b, s, self.embedding.num_embeddings)
        
        return outputs

In [142]:
class Decoder(nn.Module):
    def __init__(self,vocab_size,embedding_size , hidden_size, bidirectional=True, num_layers = 1,rnn_type='rnn',word_dropout = 0.5):
        super(Decoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers            
        
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.word_dropout = nn.Dropout(p=word_dropout)
        
        self.decoder = rnn(embedding_size, hidden_size, num_layers=num_layers, bidirectional=self.bidirectional, batch_first=True)
        self.hidden2outputs = nn.Linear(hidden_size * (2 if bidirectional else 1), hidden_size)
        self.outputs2vocab = nn.Linear(hidden_size, vocab_size)
        

    def forward(self,x,encoder_outputs,hidden,sorted_lengths,sorted_idx,batch_size):

        
        if self.bidirectional or self.num_layers > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.hidden_size)
        else:
            hidden = hidden.unsqueeze(0)

        input_embedding = self.word_dropout(x)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.tolist(), batch_first=True)
        outputs,_ = self.decoder(packed_input, hidden)
        
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _,reversed_idx = torch.sort(sorted_idx.cuda())
        padded_outputs = padded_outputs[reversed_idx]
        b,s,_ = padded_outputs.size()
        
        #print(padded_outputs.view(-1, padded_outputs.size(2)).size())
        outputs = nn.functional.log_softmax(self.outputs2vocab(self.hidden2outputs(padded_outputs.view(-1, padded_outputs.size(2)))), dim=-1)
        outputs = outputs.view(b, s, self.embedding.num_embeddings)
        
        return outputs

In [143]:
S2S=Seq2Seq(vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout,sos_idx, eos_idx, pad_idx , numpy_embedding = model_embedding, num_layers=num_layers ,rnn_type='gru',bidirectional= False).cuda()

In [144]:
NLL = torch.nn.NLLLoss(size_average=False, ignore_index = pad_idx)

In [145]:
optimizer = torch.optim.Adam(S2S.parameters(), lr=learning_rate)

In [146]:
S2S

Seq2Seq(
  (embedding): Embedding(60923, 300)
  (encoder): Encoder(
    (encoder): GRU(300, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(60923, 300)
    (word_dropout): Dropout(p=0.5)
    (decoder): GRU(300, 256, batch_first=True)
    (hidden2outputs): Linear(in_features=256, out_features=256, bias=True)
    (outputs2vocab): Linear(in_features=256, out_features=60923, bias=True)
  )
)

In [147]:
def batch(batch_size,input_var,target_var,length_var):

    shuffle_list = list(zip(input_var,target_var,length_var))
    random.shuffle(shuffle_list)
    
    start = 0
    end = batch_size
    #if len(input_var)%32 != 0:
    while end < len(input_var):
        batch_input = []
        batch_target = []
        batch_length = []
        
        batch_shuffle = shuffle_list[start:end]
        
        for i,j,k in batch_shuffle:
            batch_input.append(i)
            batch_target.append(j)
            batch_length.append(k)
            
        temp = end
        end  = end + batch_size
        start = temp
        yield batch_input, batch_target, batch_length
        
    if end >= len(input_var):
        batch_input = []
        batch_target = []
        batch_length = []
        
        batch_shuffle = shuffle_list[start:]
        
        for i,j,k in batch_shuffle:
            batch_input.append(i)
            batch_target.append(j)
            batch_length.append(k)
            
        yield batch_input, batch_target, batch_length

In [148]:
inputs_len = []
for sentence in input_index:
    inputs_len.append(len(sentence) - sentence.count(0))

In [149]:
var_index = int(len(input_index)*0.6)
test_index = int(len(input_index)*0.8)

In [32]:
epochs = 15

In [ ]:
step = 0
var_losses = []
train_losses = []
avg_losses = []
iteration = 0
for epoch in tqdm.tqdm(range(epochs)):
    
    for batch_x, batch_y, batch_len in batch(batch_size_fit, input_index[:var_index], target_index[:var_index], \
                                             inputs_len[:var_index]):
        iteration = iteration + 1
        S2S.train()
        x_ = Variable(torch.cuda.LongTensor(batch_x))
        y_ = Variable(torch.cuda.LongTensor(batch_y))
        batch_size = x_.size(0)
        length = torch.cuda.LongTensor(batch_len)

        outputs=S2S(x_,length)
        
        outputs = outputs.view(-1, outputs.size(2))
        target = y_[:, :torch.max(length)].contiguous().view(-1)
        
        NLL_loss = NLL(outputs,target)

        loss = NLL_loss/batch_size


        train_losses.append(float(loss.cpu().data))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        step += 1

        if iteration % 1000 == 0 or iteration == (len(input_index[:var_index])-1)//batch_size:
            print("Batch %04d/%i, Loss %9.4f"%( iteration, (len(input_index[:var_index])-1)//batch_size_fit, loss.data[0]))
            np.savez(L=train_losses,file='train_loss.npz')
            checkpoint_path = os.path.join(save_model_path, "E%i.pytorch"%(epoch))
            torch.save(S2S, checkpoint_path)
    iteration = 0  
    for batch_x, batch_y, batch_len in batch(batch_size_fit, input_index[var_index:test_index], target_index[var_index:test_index], \
                                             inputs_len[var_index:test_index]):
        iteration = iteration + 1
        S2S.eval()
        x_ = Variable(torch.cuda.LongTensor(batch_x))
        y_ = Variable(torch.cuda.LongTensor(batch_y))
        batch_size = x_.size(0)
        length = torch.cuda.LongTensor(batch_len)

        outputs=S2S(x_,length)
        
        outputs = outputs.view(-1, outputs.size(2))
        target = y_[:, :torch.max(length)].contiguous().view(-1)
        
        NLL_loss = NLL(outputs,target)

        loss = NLL_loss/batch_size

        var_losses.append(float(loss.cpu().data))

        if iteration % 1000 == 0 or iteration == (len(input_index[var_index:test_index])-1)//batch_size:
            print("Valid Batch %04d/%i, Loss %9.4f"%( iteration, (len(input_index[var_index:test_index])-1)//batch_size_fit, loss.data[0]))
            np.savez(L=var_losses,file='var_loss.npz')
            
    iteration = 0        
    checkpoint_path = os.path.join(save_model_path, "E%i.pytorch"%(epoch))
    torch.save(S2S, checkpoint_path)
    print("Model saved at %s"%checkpoint_path)
    print("Epoch %02d/%i, Mean ELBO %9.4f"%( epoch, epochs, np.mean(np.array(var_losses))))
    avg_losses.append(np.mean(np.array(var_losses)))
    np.savez(L=avg_losses,file='avg_losses.npz')

checkpoint_path = 'won_seq2seq\2018-Jun-27-10-26-32\E14.pytorch'

In [39]:
try:
    S2S = torch.load(checkpoint_path)
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass
S2S.cuda()
S2S.eval()


--------model not restored--------



Seq2Seq(
  (embedding): Embedding(60923, 300)
  (encoder): Encoder(
    (encoder): GRU(300, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(60923, 300)
    (word_dropout): Dropout(p=0.5)
    (decoder): GRU(300, 256, batch_first=True)
    (outputs2vocab): Linear(in_features=256, out_features=60923, bias=True)
  )
)

In [40]:
def inference(batch, hidden):
    
    hidden = hidden.view(S2S.decoder.hidden_factor, batch_size, S2S.hidden_size)
    
    t = 0
    generations=torch.cuda.FloatTensor(batch_size, max_sequence_length).fill_(pad_idx).long()
    
    while(t<max_sequence_length ):
        if t == 0:
            input_sequence = Variable(torch.Tensor(batch_size).fill_(sos_idx).long()).cuda()
        input_sequence = input_sequence.unsqueeze(1)
        input_embedding = S2S.embedding(input_sequence)
        output, hidden = S2S.decoder.decoder(input_embedding, hidden)
        
        logits = S2S.decoder.outputs2vocab(output)
        samples = torch.topk(logits,1,dim=-1)[1].squeeze()
        input_sequence = samples.squeeze()
        generations[:,t] = input_sequence.data
        if ((torch.sum(input_sequence)/input_sequence.size(0)) == eos_idx).cpu().data.numpy():
            break
        t+=1
    return generations
def print_inference(generations):


    w2i, i2w = word2index, index2word

    samples = generations.cpu().numpy()

    sent_str = [str()]*len(samples)

    for i, sent in enumerate(samples):
        for word_id in sent:
            if word_id == w2i['_PAD_']: 
                break
            elif word_id == eos_idx:
                break
            sent_str[i] += i2w[word_id] + " "
        sent_str[i] = sent_str[i].strip()
    return sent_str

def print_input(generations):


    w2i, i2w = word2index, index2word

    samples = generations.cpu().data.numpy()

    sent_str = [str()]*len(samples)

    for i, sent in enumerate(samples):
        for word_id in sent:
            if word_id == pad_idx: 
                break
            elif word_id == eos_idx:
                break
            sent_str[i] += i2w[word_id] + " "
        sent_str[i] = sent_str[i].strip()
    return sent_str

In [41]:
start = 0
end = 5

In [42]:
inputs_len = []
for sentence in input_index[start:end]:
    inputs_len.append(len(sentence) - sentence.count(0))

In [43]:
x_ = Variable(torch.cuda.LongTensor(input_index[start:end]))

batch_size = x_.size(0)
length = torch.cuda.LongTensor(inputs_len)

sorted_lengths, sorted_idx = torch.sort(length, descending=True)
input_sequence = x_[sorted_idx.cuda()]
input_embedding = S2S.embedding(input_sequence)
_,reversed_idx = torch.sort(sorted_idx.cuda())
packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.tolist(), batch_first=True)

encoder_outputs, encoder_hidden_state = S2S.encoder(packed_input,batch_size)

In [44]:
target_print = []
for i in target_index[start:end]:
    temp = []
    for j in i :
        if j == 0:
            break
        else :
            temp.append(index2word[j])
    target_print.append(temp)

In [89]:
generations=inference(batch_size, encoder_hidden_state[reversed_idx])
sent_str = print_inference(generations)
input_str = print_input(x_)
for i, j, k in zip(sent_str, input_str, target_print):
    print('************************')
    print('input: '+j)
    print("inference: "+i)
    print('target: ' + ' '.join(k[:-1]))
    print('bleu: '+str(nltk.translate.bleu_score.sentence_bleu([nltk.tokenize.word_tokenize(i)], k[:-1])))
    print('************************')

************************
input: _STA_ i thought this soup would be more like a chilli , but it is very watery . which i could tolerate as long as it still tasted good , which it does n't . i had to throw it away after only a few spoonfulls .
inference: small small small
target: no good
bleu: 0
************************
************************
input: _STA_ i just bought this soup today at my local grocer , brought it back to work with me , heated it up and almost spit out my first bite . the taste is so sweet , and it 's not just the aftertaste as a previous commenter said . this soup is horrible ! it tastes nothing like true cajun red beans and rice . i tried complaining via the healthy choice website today , only to have their servers fail twice before it wiped my complaint. < br / > < br / > do not buy this
inference: it ! the ! !
target: tastes horrible !


C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


bleu: 0.39011264866539486
************************
************************
input: _STA_ this soup is mostly broth . although it has a kick to it , it also a sweet after taste that i find pretty gross . this is the third soup i 've tried from healthy choice and they have all bed bad . it tastes nothing like real red beans and rice . i would n't recommend this soup .
inference: not much
target: not so good
bleu: 0.7598356856515925
************************
************************
input: _STA_ it is mostly broth , with the advertised 3/4 cup of vegetables . however , each cup contains about 3 _UNK_ . this is supposed to be a double serving bowl , but 1.5 tortellini is n't much of a meal . it tastes good and i like all the veggies ( plus it has low sodium ) but i really wish they would include a couple more tortellini .
inference: tastes as , as as a a a a a a a a a a a
target: where 's the tortellini ?
bleu: 0
************************
************************
input: _STA_ in the past , i

In [46]:
def batch(batch_size,input_var,target_var,length_var):
    start = 0
    end = batch_size
    #if len(input_var)%32 != 0:
    while end < len(input_var):
        batch_input = input_var[start:end]
        batch_target = target_var[start:end]
        batch_length = length_var[start:end]
        temp = end
        end  = end + batch_size
        start = temp
        yield batch_input, batch_target, batch_length
        
    if end >= len(input_var):
        batch_input  = input_var[start:]
        batch_target = target_var[start:]
        batch_length = length_var[start:]
        yield batch_input, batch_target, batch_length

In [47]:
def target_p(target_index):
    target_print = []
    for i in target_index:
        temp = []
        for j in i :
            if j == 0:
                break
            else :
                temp.append(index2word[j])
        target_print.append(temp)
    return target_print

In [48]:
inputs_len = []
for sentence in input_index:
    inputs_len.append(len(sentence) - sentence.count(0))

In [110]:
bleu_losses = []
iteration = 0
iteration = 0  
for batch_x, batch_y, batch_len in tqdm.tqdm(batch(batch_size_fit, input_index[test_index:], target_index[test_index:], \
                                         inputs_len[test_index:])):
    iteration = iteration + 1

    x_ = Variable(torch.cuda.LongTensor(batch_x))
    
    target_print = target_p(batch_y)
    
    batch_size = x_.size(0)
    length = torch.cuda.LongTensor(batch_len)

    sorted_lengths, sorted_idx = torch.sort(length, descending=True)
    input_sequence = x_[sorted_idx.cuda()]
    input_embedding = S2S.embedding(input_sequence)
    _,reversed_idx = torch.sort(sorted_idx.cuda())
    packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.tolist(), batch_first=True)

    encoder_outputs, encoder_hidden_state = S2S.encoder(packed_input,batch_size)
    
    generations=inference(batch_size, encoder_hidden_state[reversed_idx])
    sent_str = print_inference(generations)
    input_str = print_input(x_)
    
    cnt = 0
    bleu_list =[]
    bleu_result = {}
    temp={}
    for i, j, k in zip(sent_str, input_str, target_print):       
        bleu = nltk.translate.bleu_score.sentence_bleu([nltk.tokenize.word_tokenize(i)], k[:-1])
        if bleu < 1.0:
            #print("iteration: "+str(iteration)+", cnt: "+str(cnt))
            #print("input: "+j)
            temp['input'] = j
            #print("inference: "+i)
            temp['inference'] = i
            #print('target: ' + ' '.join(k[:-1]))
            temp['target'] = ' '.join(k[:-1])
            print(bleu)
        bleu_list.append(temp)
        bleu_result[cnt] = bleu_list
        with open('bleu_result.json', 'w', encoding='UTF-8') as f:
            json.dump(bleu_result, f, ensure_ascii=False)
        temp={}
        bleu_list=[]
        bleu_losses.append(bleu)
        cnt +=1

        
    loss = np.mean(np.array(bleu_losses))

    if iteration % 10 == 0 or iteration == (len(input_index[test_index:])-1)//batch_size:
        print("Valid Batch %04d/%i, Loss %9.4f"%( iteration, (len(input_index[test_index:])-1)//batch_size_fit, loss))
        np.savez(L=bleu_losses,file='bleu_loss.npz')

0it [00:00, ?it/s]C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.510029457493824
0
0
0
0


C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.4171622699855093
0.049787068367863944
0.7186082239261684
0.7952707287670506
0.7598356856515925
0.1353352832366127


C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


...

227it [02:17,  1.65it/s]

0
0.7598356856515925
0
0
0
0
0
0.5444460596606694
0.2925637512788283
0
0
0
0
0.20029051217596075
0
0
0.5623413251903491
0
0
0
0.6262844962765469
0
0
0
0
0.5773502691896257
0
0.7598356856515925
0
0.4482700320176827
0.5946035575013605
0
0.6147881529512643
0.7071067811865476
0
0.7598356856515925
0.48109772909788073
0.7598356856515925
0.36720562698935927
0
0
0.7071067811865476
0
0
0
0
0.7952707287670506
0
0.8408964152537145
0
0
0.668740304976422
0
0
0
0
0
0
0
0
0
0
0.24601580968354606


228it [02:17,  1.66it/s]

0
3.502183907414913e-06
0
0
0.5773502691896257
0
0.12068076399341783
0.8408964152537145
0.7071067811865476
0
0.668740304976422
0.3860973950960897
0
0
0
0
0.36787944117144233
0.5946035575013605
0
0
0
0
0
0
0
0
0
0.5623413251903491
0.39011264866539486
0
0
0
0
0
0
0
0.36787944117144233
0.5506953149031837
0.3081980909598119
0
0
0
0
0
0.668740304976422
0.6042750794713536
0.8408964152537145
0.7447819789879647

KeyboardInterrupt: 

In [101]:
batch_size_fit

64

In [160]:
torch.cuda.empty_cache()

In [114]:
with open('bleu_result.json', 'r', encoding='UTF-8') as f:
    target_dict=json.load(f)

In [125]:
print(target_dict['8'])

[{'input': "_STA_ i love peach tea ( and anything peach flavored ) , but this has has a cinnamon flavor that is too strong . i wanted so badly to give it 5 stars , but i would n't be honest if i did . i do n't like the cinnamon flavor . to me , peach and cinnamon do n't belong together . peach is summer and cinnamon is winter . this tea might be better in the winter as hot tea , but then again , not , because peach is too summery . cinnamon would go better with", 'inference': 'too overpowering', 'target': 'cinnamon way too strong'}]


In [126]:
target_dict.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47'])